# Projet : reconnaissance biométrique au clavier
_Un projet d'Olivier Baurain, Romain Ceccoti, Jules Micho, Baptiste Millot et Hao Zhu_

_Avril 2019_

---

L'objectif de ce notebook est de proposer une première approche à la reconnaissance comportementale avec un Rubik's Cube, voir si l'on peut discerner un utilisateur d'un autre à sa manière d'utiliser un Rubik's Cube.
Pour ce faire nous allons utiliser un alogrithme de classification issue de la bibliothèque [scikit-learn](https://scikit-learn.org/stable/index.html).

Voici les trois grandes étapes de ce notebook:

1. Récolte de données qui vont etre utilisé par notre algorithme
2. Exploitation des données
3. Interpretation de nos résultats


In [8]:
# Importation des dépendances
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

# 1. Récolte de données

Pour récolter des données d'utilisation de Rubik's Cube nous avons utilisé le cube Giiker de Xiaomi [1]   
Nous nous sommes limité a la récolte d'une séquence particulière: L'URU'LUR'U' [2]   
Pour chaque séquence nous récoltons 7 informations: le temps qui s'est écoulé entre chaque mouvement.   

Nous avons récolté les données pour 4 sujets : Baptiste, Jules, Olivier, Romain   
Chaque sujet a effectué la séquence 50 fois.   
Nous avons créé une interface web pour facilité la récolte ces données: https://olivbau.github.io/rubisen-sandbox/#/collect   

[1] http://giiker.cn/   
[2] https://en.wikipedia.org/wiki/Rubik%27s_Cube#Move_notation   

In [3]:
# Chargement du dataset
ds = pd.read_csv('DatasetLURU.csv')

In [4]:
# Affichage des premières lignes du dataset
ds.head()

,date,user,device,0,1,2,3,4,5,6,7
0,1553159921339,baptiste,Gi161124,0,1376,1140,660,962,1020,840,1259
1,1553159954516,baptiste,Gi161124,0,960,1680,661,4200,1920,2582,1078
2,1553159972516,baptiste,Gi161124,0,660,1500,720,1621,719,2343,1377
3,1553160002817,baptiste,Gi161124,0,599,1320,600,661,599,2404,477
4,1553160012118,baptiste,Gi161124,0,478,1205,896,1258,541,1199,541


In [10]:
# Affichage d'une ligne du dataset
ds.iloc[0,:]

date      1553159921339
user           baptiste
device         Gi161124
0                     0
1                  1376
2                  1140
3                   660
4                   962
5                  1020
6                   840
7                  1259
Name: 0, dtype: object

In [6]:
# Affichage des différents sujets
users = ds.user.unique()
print(users)

['baptiste' 'jules' 'olivier' 'romain']


# 2. Exploitation des données

Maintenant nous allons créer et entrainner un model grâce aux données récupérées dans la partie précédente.   
Le but de ce model est de nous dire en fonction d'une séquence fournis, quel est l'utilisateur qui l'a réalisé.   
C'est un model de classification, plus précisément nous utiliserons un algorithme SVM [3]   
Aussi nous sépérons nos données récoltées en 2: "train" et "test".   
Les données de train seront utilisé pour l'entrainement du model, alors que celles de test pour le tester.   

[3] https://scikit-learn.org/stable/modules/svm.html     

In [17]:
# Création des data de test et de train
from sklearn.model_selection import train_test_split

X = ds.drop(['date', 'user', 'device'], axis=1)
y = ds.user

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42, stratify=y)

In [20]:
# Création du model
from sklearn.svm import SVC
clf = SVC(gamma='scale', probability = True)
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# 3. Résultats

Maintenant il ne nous reste plus qu'a utiliser les données de test pour tester notre model.   

In [27]:
# Test du model
y_pred = clf.predict(X_test)

In [28]:
# Affichage de résultats
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    baptiste       0.86      0.72      0.78        25
       jules       1.00      0.96      0.98        25
     olivier       0.76      0.88      0.81        25
      romain       0.96      1.00      0.98        25

   micro avg       0.89      0.89      0.89       100
   macro avg       0.89      0.89      0.89       100
weighted avg       0.89      0.89      0.89       100



Comme nous pouvons le voir ci-dessus,   
nous obtenons des résultats plûtot satisfaisant   
Pour plus de détails le calcul et la signification de ces scores cliquez [ici](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)   
Ci dessous le détail des résultats du model


In [33]:
r = pd.DataFrame(np.zeros((users.size, users.size)), columns=users, index=users)
for index, user in enumerate(y_test.tolist()):
    r[user][y_pred[index]] += 1
print('X-axis: Model result')
print('Y-axis: Real user\n')
print(r.astype(int))

X-axis: Model result
Y-axis: Real user

          baptiste  jules  olivier  romain
baptiste        18      0        3       0
jules            0     24        0       0
olivier          7      0       22       0
romain           0      1        0      25


On peut voir que Olivier et Baptiste sont plus souvent confondu.   
On pourrait en déduire qu'ils ont des comportements similaire (du moins par rapport à nos méthodes de récoltes de données)   
Jules et Romain quant à eux semblent avoir des comportements différents des autres utilisateurs